In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# List of URLs to load documents from
urls = [
    "https://blog.adityakumar.xyz/post/use-gui-programs-in-incus-container/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [22]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [23]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.embeddings import FastEmbedEmbeddings

vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=FastEmbedEmbeddings()
)
# return top 4 most relevant documents
retriever = vectorstore.as_retriever(k=4)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [24]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"]
)

In [29]:
llm = ChatOllama(
    model="llama3.2",
    temperature=0
)

In [30]:
rag_chain = prompt | llm | StrOutputParser()

In [31]:
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain

    def run(self, question):
        documents = self.retriever.invoke(question)
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [32]:
rag_application = RAGApplication(retriever, rag_chain)
question = "How to run GUI programs in incus container?"
answer = rag_application.run(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: How to run GUI programs in incus container?
Answer: To run GUI programs in Incus Container, create a GUI profile using the command "incus profile create gui" and edit it with "incus profile edit gui". Add your container to the GUI profile using "incus profile add <container> gui". Then, install an X server inside the container using commands like "yay -S xorg-server xorg-xauth xorg-drivers mesa-utils" for Arch Linux.
